In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
with open("Smogon's Official Ladder Tournament VIII - Replay Thread _ Smogon Forums.html","r") as html_file:
    content = html_file.read()
    
soup = BeautifulSoup(content, 'lxml')

In [3]:
usernames = soup.find_all('a', class_="username")
unames = []
for names in usernames:
    names = names.text.replace('','')
    if names != 'Wigglytuff'and names != "Vileman":
        unames.append(names)

In [4]:
setnames = set(unames)
finalnames = list(setnames)
print(finalnames)

['Insult', 'Ahsan-219', 'SoulWind', 'Tace', 'MichaelderBeste2', 'haxrme', 'Rubyblood', 'eclipseY', 'Eternal Spirit', 'Gray', 'Lusa', 'Ayoukoo', 'Misterioussaint', 'beatiful', 'Lord_Enz', 'SpookyZ', 'Bushtush', 'Chaitanya', 'Tricking', 'John W', 'devin', 'BIHI', 'HSA', 'UnzipsCrogre', 'TDNT', 'Sagiri', 'ABR', 'Storm Zone', 'Flex OKLM', 'Punny', 'Ox the Fox', 'Empo']


In [5]:
linksoup = soup.find_all('a', class_= 'link link--external')

In [6]:
links_temp = []
for x in linksoup:
    links_temp.append(x['href'])
    
finallinks = []
for x in links_temp:
    if 'replay' in x:
        finallinks.append(x)

In [7]:
def matchPlayerID(name):
    if(name == 'Sagiri I.' or name == 'Sagiri I'):
        return 'Sagiri'
    elif(name == 'Lopunny kicks'):
        return "Punny"
    elif(name == 'devwin'):
        return 'devin'
    elif(name == 'Tace㋛'):
        return 'Tace'
    elif(name == 'SCILLATRON9000' or name == 'SCILLATRON 9000' or name == 'uhuhuhu2'):
        return 'Tricking'
    elif(name == 'INSULT'):
        return 'Insult'
    elif(name == 'Ayouko'):
        return 'Ayoukoo'
    elif(name == 'BIHI Gregson' or name == 'BIHI Asogi' or name == 'Cotisation URSSAF'):
        return 'BIHI'
    elif(name == 'EclipseY'):
        return 'eclipseY'
    elif(name == 'misterioussaint'):
        return 'Misterioussaint'
    elif(name == 'hsa'):
        return 'HSA'
    elif(name == 'imjustgray'):
        return 'Gray'
    elif(name == 'LordEnz'):
        return 'Lord_Enz'
    elif(name == 'Bbeeaa' or name == 'FREE XIMRAPTOR' or name == 'Freeximraptor'):
        return 'beatiful'
    elif(name == 'SOULWIND'):
        return 'SoulWind'
    elif(name == 'john w'):
        return 'John W'
    elif(name == 'storm zone'):
        return 'Storm Zone'
    else:
        return name

In [ ]:
import requests
edges = []
for x in finallinks:
    battlefile = requests.get(x)
    temp_soup = BeautifulSoup(battlefile.text, 'lxml')
    players = []
    for y in temp_soup.find_all('a', class_ = 'subtle'):
        players.append(y.text.replace('',''))
    logs = battlefile.text.splitlines()
    winline = ""
    for i in logs:
        if "win" in i:
            winline = i
    if players[0] in winline:
        edges.append([matchPlayerID(players[1]) , matchPlayerID(players[0])]) 
    elif players[1] in winline:
        edges.append([matchPlayerID(players[0]) , matchPlayerID(players[1])]) 
    

In [ ]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from(finalnames)
G.add_edges_from(edges)


### Using PageRank

In [ ]:
PRank = nx.pagerank(G, alpha=0.85, personalization=None, weight='weight', dangling=None)

In [ ]:
d1 = sorted(PRank.items(), key=lambda x: x[1])

In [ ]:
df = pd.DataFrame(data = d1, columns=['Player','Score'])
print(df)
df.to_excel('PR.xlsx')


### Using Win/Loss Ratio

In [ ]:
dictWin = {i : 0 for i in finalnames}
dictLoss = {i : 0 for i in finalnames}
for x in edges:
    dictLoss[x[0]] = dictLoss[x[0]] + 1
    dictWin[x[1]] = dictWin[x[0]] + 1
    

In [ ]:
dictWinLoss = {i : 0 for i in finalnames}
for key in dictWinLoss:
    dictWinLoss[key] = dictWin[key]/dictLoss[key]

In [ ]:
d2 = sorted(dictWinLoss.items(), key=lambda x: x[1])

In [ ]:
df = pd.DataFrame(data = d2, columns=['Player','Score'])
print(df)
df.to_excel('WL.xlsx')